In [1]:
#Digit reader using MNIST and convolution neural networks
import pickle
import gzip
import numpy as np
from scipy.ndimage import imread
import cv2
import sklearn.utils

DATA_SET = 'mnist.pkl.gz'

In [2]:
from keras.utils import to_categorical
#Data modification
f = gzip.open(DATA_SET, 'rb')
train_set, validation_set, test_set = pickle.load(f, encoding = 'latin1')
f.close()

train_x = train_set[0]
train_y = train_set[1]
test_x = test_set[0]
test_y = test_set[1]
val_x = validation_set[0]
val_y = validation_set[1]

#Resize train_x and test_x to be square np arrays instead of flat
train_x.resize((len(train_x), 28, 28, 1))
test_x.resize((len(test_x), 28, 28, 1))
val_x.resize((len(val_y), 28, 28, 1))

#Categorize Y data
train_y_cat = to_categorical(train_y, num_classes=10)
test_y_cat = to_categorical(test_y, num_classes=10)
val_y_cat = to_categorical(val_y, num_classes=10)
print (train_x.shape)
print (train_y_cat.shape)

validation_data = (val_x, val_y_cat)

print (train_x[7][14][14])

Using TensorFlow backend.


(50000, 28, 28, 1)
(50000, 10)
[ 0.98828125]


In [3]:
#Create convolutional neural net from keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Activation, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras import optimizers
from keras import regularizers

In [4]:
model = Sequential()
print (train_x.shape)

model.add(Conv2D(input_shape = [28, 28, 1], filters = 128, kernel_size=3, 
                activation='relu', data_format="channels_last"))

model.add(Conv2D(filters = 128, kernel_size=3,
                activation='relu'))

model.add(MaxPooling2D(pool_size=2))

model.add(Conv2D(filters = 64, kernel_size=2,
                activation='relu'))
model.add(Conv2D(filters = 64, kernel_size=2,
                activation='relu'))

model.add(Flatten())

model.add(Dense(128))

model.add(Dense(10, activation='softmax'))

optimizer = 'RMSProp'
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])

(50000, 28, 28, 1)


In [5]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 128)       1280      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 11, 11, 64)        32832     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 10, 10, 64)        16448     
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               819328    
__________

In [6]:
#Some hyper parameters to tune
EPOCHS = 5
BATCH_SIZE = 100

model.fit(train_x, train_y_cat, epochs=EPOCHS, batch_size=BATCH_SIZE,
         validation_data=validation_data, verbose=True, shuffle=True) #Check to use validation data

Train on 50000 samples, validate on 10000 samples
Epoch 1/5
50000/50000 [==============================] - 68s 1ms/step - loss: 0.0285 - acc: 0.9903 - val_loss: 0.0144 - val_acc: 0.9955
Epoch 2/5
50000/50000 [==============================] - 71s 1ms/step - loss: 0.0090 - acc: 0.9970 - val_loss: 0.0074 - val_acc: 0.9976
Epoch 3/5
50000/50000 [==============================] - 68s 1ms/step - loss: 0.0062 - acc: 0.9980 - val_loss: 0.0073 - val_acc: 0.9976
Epoch 4/5
50000/50000 [==============================] - 72s 1ms/step - loss: 0.0047 - acc: 0.9985 - val_loss: 0.0070 - val_acc: 0.9979 2s - loss: 0
Epoch 5/5
50000/50000 [==============================] - 61s 1ms/step - loss: 0.0038 - acc: 0.9988 - val_loss: 0.0071 - val_acc: 0.9980


In [8]:
#Evaluate the efficacy of the model
loss, acc = model.evaluate(test_x, test_y_cat, 
                           batch_size=BATCH_SIZE)
print('Loss of ')
print('')
print('Accuracy of %.2f%% ' % (acc * 100.))


10000/10000 [==============================] - 4s 351us/step
Loss of 

Accuracy of 99.85% 


In [43]:
predictions = model.predict(test_x)
import matplotlib.pyplot as plt
def make_number(pred):
    for ind in range(len(pred)):
        if pred[ind] > .6:
            return ind
print (predictions[0])

[  2.65754107e-12   1.41304234e-11   7.66644845e-08   1.41609062e-08
   4.31656291e-15   1.77998973e-13   5.97035805e-17   9.99999881e-01
   2.61604644e-11   1.82759197e-10]


In [45]:
#Print pictures it is getting wrong

for i in range(400):
    if(make_number(predictions[i]) != test_y[i]):
        print (("Image ") + str(i+1))
        print ("Guessed it was: ") + str((make_number(predictions[i])))
        print ("Was actually: ") + str(test_y[i])
        plt.imshow(test_x[i].reshape((28, 28)), cmap=plt.get_cmap('gray'))
        plt.show()

Image 322
Guessed it was: 


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'